In [1]:
import tensorflow as tf
import numpy as np

在下面的代码中，卷积是使用tf.nn.conv2d, 池化使用tf.nn.max_pool,下面来详细的讲解一下这两个函数的用法。

### tf.nn.conv2d

这个函数的功能是：给定4维的input和filter，计算出一个2维的卷积结果。函数的定义为：

def conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None,
           data_format=None, name=None):

前几个参数分别是input, filter, strides, padding, use_cudnn_on_gpu, …

下面来一一解释 

input：待卷积的数据。格式要求为一个张量，[batch, in_height, in_width,in_channels]. 分别表示 批次数，图像高度，宽度，输入通道数。 

filter： 卷积核。格式要求为[filter_height, filter_width, in_channels, out_channels]. 分别表示 卷积核的高度，宽度，输入通道数，输出通道数。 

strides :一个长为4的list. 表示每次卷积以后卷积窗口在input中滑动的距离 

padding ：有SAME和VALID两种选项，表示是否要保留图像边上那一圈不完全卷积的部分。如果是SAME，则保留 

use_cudnn_on_gpu ：是否使用cudnn加速。默认是True

### tf.nn.max_pool 

进行最大值池化操作,而avg_pool 则进行平均值池化操作.函数的定义为：

def max_pool(value, ksize, strides, padding, data_format="NHWC", name=None):



value: 一个4D张量，格式为[batch, height, width, channels]，与conv2d中input格式一样 

ksize: 长为4的list,表示池化窗口的尺寸 

strides: 池化窗口的滑动值，与conv2d中的一样 

padding: 与conv2d中用法一样。

第一个参数value：需要池化的输入，一般池化层接在卷积层后面，所以输入通常是feature map，依然是[batch, height, width, channels]这样的shape


第二个参数ksize：池化窗口的大小，取一个四维向量，一般是[1, height, width, 1]，因为我们不想在batch和channels上做池化，所以这两个维度设为了1

第三个参数strides：和卷积类似，窗口在每一个维度上滑动的步长，一般也是[1, stride,stride, 1]

第四个参数padding：和卷积类似，可以取'VALID' 或者'SAME'

返回一个Tensor，类型不变，shape仍然是[batch, height, width, channels]这种形式


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("E:/machine_data/digit_recongition/tensorflow/",one_hot=True)

Extracting E:/machine_data/digit_recongition/tensorflow/train-images-idx3-ubyte.gz
Extracting E:/machine_data/digit_recongition/tensorflow/train-labels-idx1-ubyte.gz
Extracting E:/machine_data/digit_recongition/tensorflow/t10k-images-idx3-ubyte.gz
Extracting E:/machine_data/digit_recongition/tensorflow/t10k-labels-idx1-ubyte.gz


In [3]:
#定义权重和偏置
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # 变量的初始值为截断正太分布（即相当于正太分布中截取一段出来）
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [4]:
def conv2d(x, W):
    """
    tf.nn.conv2d功能：给定4维的input和filter，计算出一个2维的卷积结果
    前几个参数分别是input, filter, strides, padding, use_cudnn_on_gpu, ...
    input   的格式要求为一个张量，[batch, in_height, in_width, in_channels],批次数，图像高度，图像宽度，通道数
    filter  的格式为[filter_height, filter_width, in_channels, out_channels]，滤波器高度，宽度，输入通道数，输出通道数
    strides 一个长为4的list. 表示每次卷积以后在input中滑动的距离
    padding 有SAME和VALID两种选项，表示是否要保留不完全卷积的部分。如果是SAME，则保留
    use_cudnn_on_gpu 是否使用cudnn加速。默认是True
    data_frame:默认格式为"NHWC", 数据按这样的顺序存储：[batch, in_height, in_width, in_channels]，也可以用这种方式："NCHW", 数据按这样的顺序存储：
    [batch, in_channels, in_height, in_width
    对于strides的说明：
    input 中的每个patch都作用于filter，每个patch都能获得其他patch对filter的训练需要满足strides[0] = strides[3] = 1
    大多数水平步长和垂直步长相同的情况下：strides = [1, stride, stride, 1]
    """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [5]:
def max_pool_2x2(x):
    """
    tf.nn.max_pool 进行最大值池化操作,而avg_pool 则进行平均值池化操作
    几个参数分别是：value, ksize, strides, padding,
    value:  一个4D张量，格式为[batch, height, width, channels]，与conv2d中input格式一样
    ksize:  长为4的list,表示池化窗口的尺寸
    strides: 窗口的滑动值，与conv2d中的一样
    padding: 与conv2d中用法一样。
    """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [6]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1]) #将输入按照 conv2d中input的格式来reshape，reshape

In [7]:
"""
# 第一层
# 卷积核(filter)的尺寸是5*5, 通道数为1，输出通道为32，即feature map 数目为32
# 又因为strides=[1,1,1,1] 所以单个通道的输出尺寸应该跟输入图像一样。即总的卷积输出应该为?*28*28*32
# 也就是单个通道输出为28*28，共有32个通道,共有?个批次
# 在池化阶段，ksize=[1,2,2,1] 那么卷积结果经过池化以后的结果，其尺寸应该是？*14*14*32
"""
W_conv1 = weight_variable([5, 5, 1, 32])  # 卷积是在每个5*5的patch中算出32个特征，分别是patch大小，输入通道数目，输出通道数目
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.elu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


In [8]:
"""
# 第二层
# 卷积核5*5，输入通道为32，输出通道为64。
# 卷积前图像的尺寸为 ?*14*14*32， 卷积后为?*14*14*64
# 池化后，输出的图像尺寸为?*7*7*64
"""
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.elu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [9]:
# 第三层 是个全连接层,输入维数7*7*64, 输出维数为1024
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.elu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32) # 这里使用了drop out,即随机安排一些cell输出值为0，可以防止过拟合
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 第四层，输入1024维，输出10维，也就是具体的0~9分类
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2) # 使用softmax作为多分类激活函数
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1])) # 损失函数，交叉熵，
#reduction_indices在哪一维上求解，0：列，1：行，None:所有元素求均值
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) # 使用adam优化
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1)) # 计算准确度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables()) # 变量初始化
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        # print(batch[1].shape)
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.1
step 100, training accuracy 0.86
step 200, training accuracy 0.9
step 300, training accuracy 0.94
step 400, training accuracy 0.92
step 500, training accuracy 1
step 600, training accuracy 0.96
step 700, training accuracy 0.92
step 800, training accuracy 0.96
step 900, training accuracy 0.94
step 1000, training accuracy 0.96
step 1100, training accuracy 0.9
step 1200, training accuracy 0.86
step 1300, training accuracy 1
step 1400, training accuracy 0.96
step 1500, training accuracy 0.92
step 1600, training accuracy 0.96
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 0.98
test accuracy 0.9726


In [10]:
batch[0]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [11]:
batch[1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [12]:
batch[1].shape

(50, 10)

### 查看网络中间结果

In [13]:
# 我们先来看看数据是什么样的
img1 = mnist.train.images[1]
label1 = mnist.train.labels[1]
print (label1)  # 所以这个是数字 5 的图片
print ('img_data shape =', img1.shape)  # 我们需要把它转为 28 * 28 的矩阵
img1.shape = [28, 28]

[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
img_data shape = (784,)


In [14]:
import matplotlib.pyplot as plt
# import matplotlib.image as mpimg  # 用于读取图片，这里用不上

print (img1.shape)
plt.imshow(img1)
plt.axis('off') # 不显示坐标轴
plt.show() 

(28, 28)


In [15]:
# 我们可以通过设置 cmap 参数来显示灰度图
plt.imshow(img1, cmap='gray') # 'hot' 是热度图
plt.show()

In [ ]:
# 首先应该把 img1 转为正确的shape (None, 784)
X_img = img1.reshape([-1, 784])
y_img = mnist.train.labels[1].reshape([-1, 10])

In [24]:
# 我们要看 Conv1 的结果，即 h_conv1
result = h_conv1.eval(feed_dict={x: X_img, y_: y_img, keep_prob: 1.0})
print (result.shape)
print (type(result))

(1, 28, 28, 32)
<class 'numpy.ndarray'>


In [26]:
#显示经过第一层卷积后结果
for _ in range(32):
    show_img = result[:,:,:,_]
    show_img.shape = [28, 28]
    plt.subplot(4, 8, _ + 1)
    plt.imshow(show_img, cmap='gray')
    plt.axis('off')
plt.show()

从上面的结果中，我们可以看到不同的滤波器（卷积核）学习到了不同的特征。比如第一行中，第一个滤波器学习到了边缘信息，第4个卷积核，则学习到了骨干的信息

In [28]:
# 我们要看 Conv2 的结果，即 h_conv1
result2 = h_conv2.eval(feed_dict={x: X_img, y_: y_img, keep_prob: 1.0})
print (result2.shape)
print (type(result2))

(1, 14, 14, 64)
<class 'numpy.ndarray'>


In [30]:
for _ in range(64):
    show_img = result2[:,:,:,_]
    show_img.shape = [14, 14]
    plt.subplot(8, 8, _ + 1)
    plt.imshow(show_img, cmap='gray')
    plt.axis('off')
plt.show()

好吧，经过两层卷积后的特征已经只能看到模糊的轮廓了